<h1 align = "center">Developing a Scheduling Application in QGIS & Visual Basic</h1>
<p>&nbsp;</p>

<div align = "center">
<img src="meter.jpg" width="300"/>
</div>
<p>&nbsp;</p>


<p>I recently worked for a company that read electric, water, and solar meter routes within a 2,236 mi<sup>2</sup> service area in the Pacific Northwest. The company read 534 routes per month in total, with 81 routes in progress, and an additional 27 routes made active on an average day. Routes had to be read in their entirety within 4 business days. The company was able to work overtime on Saturdays to get caught up on routes that were currently in progress, but overtime was costly and often led to burnout and high turnover. Due to the size of the service area, deadline for route completion, and the cost of working overtime, the logistics of the operation were a critical consideration, with special attention given to the scheduling of employees.</p>


<p>At the end of a given day, each Manager was responsible for scheduling their 7-person team and up to 3 floats to cover 18 routes that were currently in progress and 9 routes that would become active on the following day. The data provided for scheduling included: route age (day 0 to 3 corresponding to the 4-day deadline), the slot number and name of the employee assigned to the route, the route number, the total count (i.e. total number of meters on the full route), current count (i.e. number of meters that remained to be read), and the estimated time it should take to complete the full route (to the nearest half hour).</p>

<table>

<tr><th> Day </th><th> Slot </th><th> Reader </th><th> Route </th><th> Est Time</th><th>Total Count </th><th> Current Count <th></tr>
<tr><td style="text-align:center"> 2   </td><td style="text-align:center"> 1    </td><td style="text-align:center"> Kyle   </td><td style="text-align:center"> 1223  </td><td style="text-align:center"> 6.5  </td><td style="text-align:center"> 375 </td><td style="text-align:center"> 181 <td></tr>
    
<tr><td style="text-align:center"> 2   </td><td style="text-align:center"> 2    </td><td style="text-align:center"> Stan   </td><td style="text-align:center"> 3523  </td><td style="text-align:center"> 5.5    </td><td style="text-align:center"> 208 </td><td style="text-align:center">93<td></tr>
    
<tr><td style="text-align:center"> 2   </td><td style="text-align:center"> 3    </td><td style="text-align:center"> Kenny  </td><td style="text-align:center"> 1323  </td><td style="text-align:center"> 7.5    </td><td style="text-align:center"> 147 </td><td style="text-align:center">72<td></tr>
    
<tr><td style="text-align:center"> 2   </td><td style="text-align:center"> 4    </td><td style="text-align:center"> Eric   </td><td style="text-align:center"> 2623  </td><td style="text-align:center"> 9.5 </td><td style="text-align:center"> 363 </td><td style="text-align:center">197<td></tr>
    
<tr><td style="text-align:center"> 2   </td><td style="text-align:center"> 5    </td><td style="text-align:center"> Wendy  </td><td style="text-align:center"> 3123  </td><td style="text-align:center"> 3.5    </td><td style="text-align:center"> 615 </td><td style="text-align:center">468<td></tr>
    
<tr><td style="text-align:center"> 2   </td><td style="text-align:center"> 6    </td><td style="text-align:center"> Butters</td><td style="text-align:center"> 2423  </td><td style="text-align:center"> 4.5  </td><td style="text-align:center"> 187 </td><td style="text-align:center">39<td></tr>
    
<tr><td style="text-align:center"> 2   </td><td style="text-align:center"> 7    </td><td style="text-align:center"> Timmy  </td><td style="text-align:center"> 2723  </td><td style="text-align:center"> 8.5    </td><td style="text-align:center"> 481 </td><td style="text-align:center">453<td></tr>
    
</table>
   
<p>Based on these numbers, Managers could figure out the remaining time needed to complete a route, calculate the percentage of the total time remaining for each route, or calculate the percentage of meters remaining to be read. They could also sum or average each of these numbers to get an overview for their team. Managers also had access to a map of the routes, which they could use to roughly estimate the drive time to a route, or determine which employees would be working close enough to a high priority route to provide assistance. However, Managers were not trained to produce these additional metrics, so they would mostly rely on the current count for scheduling, following the logic that a higher count indicated more work and more work was a priority over less work. This approach to scheduling proved problematic because the estimated time to complete a route was not strongly correlated with the total count (r = 0.19), and drive times to, from, and between routes often significantly reduced the amount of time available to employees to read their routes, resulting in lower productivity than Managers expected.</p>    
<p>&nbsp;</p>

<div align = "center">
<img src="corr1.png" width = "432"/>
</div>
<p>&nbsp;</p>

<p>My goal for this project was to develop an application for managers that provided: (1) relevant dashboard KPI's for their teams (e.g. % of work hours remaining), (2) a schedule builder that included drive times and other relevant route information, (3) hourly estimates for employee location and number of meters read (productivity) based on the schedule, and (4) daily estimates for the number of meters read for each employee and team. I used the ORS Tools plugin and OpenStreetMaps in QGIS to estimate drive times to routes and between routes. I created the dashboard in Tableau and schedule builder and location/meter count estimator using Excel Visual Basic for Applications (VBA).</p>

<h3 align = "left"> Calculating Drive Times in QGIS</h3>

<p>I needed to calculate the drive time from our fleet lot to each one of our 534 routes. I also needed to calculate drive times between routes that were active within the each 4-day window, which included 48,962 pairings. To get those calculations, I needed a starting point and a destination. The starting point was our fleet lot, so that wasn't difficult, but the destination should be the location where an employee would start each route, and that was complicated for a couple of reasons.</p> 

<p>An employee would likely drive to the address for the first meter on a route if the route was full (newly activated), but that wasn't mandatory and it wasn't uncommon for an experienced employee to start in a different location depending on what they knew of the route. For a route in progress, an employee would start where they left off the previous day. If two or more employees were working on a route, they wouldn't start at the same address, and if an employee came from another route to help they would likely start at the middle of what remained on the route and work forward, or start at the back of the route and work backwards. So, there wasn't a definitive destination/starting point for each day the route was active, or for each employee that worked on the route.</p>

<p>My initial solution to this problem was to use a central location within each route as a surrogate destination. In this case, I used the centroid (mean of the x,y coordinates for the route geometry) as my central location. After multiple errors popped when running ORS Tools, I discovered that many of the centroids had ended up in the middle of a field, lake, or forest. ORS Tools will not calculate drive times for locations that are further than 350m from a road, so this approach didn't work. </p>

<p>My next solution was to find the average drive time to all of the points where the boundary of a route intersected with a public road. This might have been the best solution if I had significantly more RAM, or if the ORS Tools API allowed unlimited queries. As this was not the case, calculating ~12,000 drive times to get the average for each route would have taken at least 6 days and would not have been a feasible option for finding the average drive time between routes (48,962 pairs). The solution I went with was to select the route/road intersection point closest to the fleet lot as the surrogate destination location for each route. The process for this started with getting shaprefiles for the routes and roads in our service area.</p>

<h3 align = "left"> Starting Point</h3>

<p>My starting point for the project was finding a shapefile for all of the routes within our service area. If I couldn't get the shapefile, I was going to have to georeference and digitize a route map we had in the office, which would have been time consuming and painfull. But, I was aware that a shapefile existed for all of the routes in our service area because I knew the map hanging on the wall in our office was created using a shapefile. So, I emailed the GIS folks at our client's office and they gracefully emailed me the shapefile (METER_AR_SH) displayed in the image below.</p>

<p>&nbsp;</p>
<div align ="center" >
<img src="All_Routes.png" width="800" />
</div>
<p>&nbsp;</p>

<p>We used a 4-digit number to identify our routes, which corresponded to the last 4 digits in the METER_READ column in the attribute table for the METER_AR_SH shapefile. To select our company's routes from the overall route shapefile, I needed to join a CSV table (ROUTE) that contained our route information with the attribute table for the METER_AR_SH shapefile.</p>
<p>&nbsp;</p>

<div align ="center">
<img src="All_Routes_AT.png" width="800"/>
</div>
<p>&nbsp;</p>

<p>To create the join, I needed a common field in each table as the basis for the join. To create a common field for the join, I used the Field Calculator in QGIS to create a new "Route" field in the attribute table for the ROUTE_AR_SH shapefile, and I populated the field with the right 4 digits from the METER_READ column.</p>
<p>&nbsp;</p>

<div align = "center">
<img src="All_Routes_Add_Field.png" width="800"/>
</div>
<p>&nbsp;</p>

<p>The attribute table for the METER_AR_SH shapefile now had a common field (ROUTE) I could use for the join.</p>
<p>&nbsp;</p>

<div align = "center">
<img src="All_Routes_Field_Added.png" width="800"/>
</div>
<p>&nbsp;</p>

<p>The ROUTE table also included several fields I wanted to attach to the geometry for our routes (e.g. team, slot, cycle, estimated time). In the join, these fields were only populated for our company's routes, which provided a simple means of differentiating our routes from our clients routes (image below). </p>
<p>&nbsp;</p>

<div align = "center">
<img src="Route_Table.png" width="800"/>
</div>
<p>&nbsp;</p>

<p>Joining the two tables using the common "Route" field.
<p>&nbsp;</p>

<div align = "center">
<img src="All_RT_Join.png" width="800"/>
</div>
<p>&nbsp;</p>

<p>Because the ROUTE_TEAM field was only populated for our company's routes in the joined table, I was able to use the field to select our 534 routes using the simple Select by Expression below.
<p>&nbsp;</p>

<div align = "center">
<img src="All_Select_By.png" width="800"/>
</div>
<p>&nbsp;</p>

<p>I saved the selected routes as a new vector layer (i.e. polygon shapefile) entitled, "Company_Routes".</p>
<p>&nbsp;</p>

<div align = "center">
<img src="Company_Routes.png" width="659"/>
</div>
<p>&nbsp;</p>

<p>Company_Routes shapefile (blue) overlying the original METER_AR_SH (white).</p>
<p>&nbsp;</p>

<div align = "center">
<img src="C_Routes_Shp.png" width="680"/>
</div>
<p>&nbsp;</p>

<p>To find the points where the boundaries of each route intersect with roads, I needed to first convert the geometry of Company_Routes layer from polygons to lines. You can intersect lines with lines, or polygons with polygons, but you can't intersect polygons with lines.</p>
<p>&nbsp;</p>

<div align = "center">
<img src="Routes_To_Lines.png" width="800"/>
</div>
<p>&nbsp;</p>

<p>Company_Routes coverted from polygons to lines.</p>
<p>&nbsp;</p>
<div align = "center">
<img src="Route_Lines.png" width="800"/>
</div>
<p>&nbsp;</p>

<p>I also needed a shapefile for the roads in our coverage area. Our service area included one whole county and part of another county. That meant that I needed a roads shapefile for each county, and I needed to extract the partial county roads and merge them with the whole county roads. I downloaded Tiger/Line shapefiles for the county roads from the US Census Bureau, available here: https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.html</p>
<p>&nbsp;</p>

<p>Road layers for each county.</p>
<p>&nbsp;</p>

<div align = "center">
<img src="All_Roads.png" width="800"/>
</div>
<p>&nbsp;</p>

<p>I used the select by polygon tool to select the roads I needed from the partial county in our service area.</p>
<p>&nbsp;</p>

<div align = "center">
<img src="Cam_Selection.png" width="800"/>
</div>
<p>&nbsp;</p>

<p>I created a vector layer (i.e. lines shapefile) from the selection above.</p>
<p>&nbsp;</p>

<div align = "center">
<img src="Sno_Cam_Roads.png" width="800"/>
</div>
<p>&nbsp;</p>

<p>Next, I needed to merge the two roads layers together.</p>
<p>&nbsp;</p>

<div align = "center">
<img src="Merged_Roads.png" width="800"/>
</div>
<p>&nbsp;</p>

<p>The result of the merge was one roads layer for the entire service area.</p>
<p>&nbsp;</p>

<div align = "center">
<img src="All_Roads_Merged.png" width="800"/>
</div>
<p>&nbsp;</p>

<p>I needed to intersect the roads layer (lines) with the Company_Routes layer (lines) to create a vector layer (i.e. points shapefile) that contained a point for every intersection between the roads and routes layers.</p>
<p>&nbsp;</p>

<div align = "center">
<img src="Routes_Roads_Intersect.png" width="800"/>
</div>
<p>&nbsp;</p>

<p>Intersection points between the roads and routes layers.</p>
<p>&nbsp;</p>

<div align = "center">
<img src="Near_Pts.png" width="750"/>
</div>
<p>&nbsp;</p>

<p>Next, I needed to find the intersection point for each route that was closest to our fleet lot. My intention was to just measure the straight line distance, but this created a bit of a problem, due to the island-turned-peninsula on the western edge of our service area.</p> 

<p>Employees accessed the island from its northeast corner, which meant that the northeastern most intersection point for each route would be closest (in drive time) to the fleet lot for the majority of routes on the island, but because of the size of the island and its north-south orientation, the southeastern most intersection point for each route would be the closest (in distance) to the fleet lot.</p>

<p>To fix this problem, I separated the island intersection points from the rest, and calculated the distance from each point to the access point in the northeast corner of the island.</p>
<p>&nbsp;</p>

<div align = "center">
<img src="Near_Pts_Sep.png" width="800"/>
</div>
<p>&nbsp;</p>

<p>Calculating the distance from the island intersection points to the access point for the island.</p>
<p>&nbsp;</p>

<div align = "center">
<img src="Pt_Distances.png" width="800"/>
</div>
<p>&nbsp;</p>

<p>Distance from each island intersection point to the island access point.</p>
<p>&nbsp;</p>

<div align = "center">
<img src="Distance.png" width="800"/>
</div>
<p>&nbsp;</p>

<p>For the next step, I needed to join the point distance shapefile with the intersection points shapefile to get the route field in the same table as the calculated distances.</p>
<p>&nbsp;</p>

<div align = "center">
<img src="Int_Dist_Join.png" width="800"/>
</div>
<p>&nbsp;</p>

<p>Once the tables were joined, I was able to write a SQL query to select the intersection point closest to the access point for each route on the island.</p>
<p>&nbsp;</p>

<div align = "center">
<img src="SQL.png" width="800"/>
</div>
<p>&nbsp;</p>

<p>Now, I needed to join the nearest point attribute table with the intersection points attribute table to attach all of the route fields to the nearest points.</p>
<p>&nbsp;</p>

<div align = "center">
<img src="Int_Near_Join.png" width="800"/>
</div>
<p>&nbsp;</p>

<p>Because the distance field was only populated for the nearest points in the join with the intersection points attribute table, I was able to use that field to select all of the nearest points together with their route fields.</p>
<p>&nbsp;</p>

<div align = "center">
<img src="Select_Pts.png" width="800"/>
</div>
<p>&nbsp;</p>

<p>The resulting vector layer with the route boundary/roads intersection points that were closest to the access point for the island and our fleet lot.</p>
<p>&nbsp;</p>

<div align = "center">
<img src="Camano_St_Pts.png" width="800"/>
</div>
<p>&nbsp;</p>

<p>I then repeated the previous 7 steps for the remainder of my intersection points using the fleet lot to measure distance and determine the closest point. After that, I merged the shapefiles to create a single shapefile with all of the surrogate destination locations for each route.</p>
<p>&nbsp;</p>

<div align = "center">
<img src="All_Start_Pts.png" width="756"/>
</div>
<p>&nbsp;</p>

<p>Finally, I used ORS Tools to calculate the fastest drive time to the destination locations for each route.</p>
<p>&nbsp;</p>

<div align = "center">
<img src="ORS Tools.png" width="800"/>
</div>
<p>&nbsp;</p>

<p>The DURATION_H field shows the drive times to each route in hours.</p>
<p>&nbsp;</p>

<div align = "center">
<img src="Drive_Time_Data.png" width="700"/>
</div>
<p>&nbsp;</p>

<p>The fastest path to each route.</p>
<p>&nbsp;</p>

<div align = "center">
<img src="Drive_Time.png" width="800" />
</div>
<p>&nbsp;</p>

<p>Fastest path and drive time from ORS Tools for a selection of 26 routes.</p>
<p>&nbsp;</p>

<div align = "center">
<img src="DTR.png" width="800"/>
</div>
<p>&nbsp;</p>

<p>From this point, the process to calculate drive times between routes was just a matter of determining which routes were active at the same time and using ORS Tools to calculate the drive time for each pairing, which concluded the GIS portion of the project. The final step was to add the drive time for each route to the ROUTE table, and create a second table (TIME_BTWN) for drive times between routes.</p> 
<p>&nbsp;</p>

<div align = "center">
<img src="Route_Table_End_GIS.png" width="724" />
</div>
<p>&nbsp;</p>

<p>I needed a unique ID for index/match or lookup functions in the TIME_BTWN table, so I added a "Sort" column, which contains a concatenation of the higher value route number and lower value route number separated by a hyphen. I used the following function to lookup drive times when two routes were entered into the schedule:</p>
<p>&nbsp;</p>

<p>=IF(O3<>"",ROUND(INDEX(TIME_BTWN!A2:B48963,MATCH((CONCAT(MAX(K3,O3),"-",MIN(K3,O3))),TIME_BTWN!A2:A48963,0),2),0),"")</p>
<p>&nbsp;</p>

<div align = "center">
<img src="Time_Btwn.png" width="578"/>
</div>
<p>&nbsp;</p>
    
<h3 align = "left"> Dashboard</h3>

<p>I originally created the dashboard in Excel, but moved everything over to Tableau to take advantage of the spatial analysis features Tableau offers. Tableau allowed me to include an interactive map in the Dashboard, which could be used by Managers while scheduling. The dashboard can be viewed here: https://public.tableau.com/views/SchedulingDashboard_16372903314670/TCDashboard?:language=en-US&:embed=y&:embed_code_version=3&:loadOrderID=0&:display_count=y&publish=yes&:origin=viz_share_link</p> 

<div class='tableauPlaceholder' id='viz1637291043343' style='position: relative'><noscript><a href='#'><img alt='TC Dashboard ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Sc&#47;SchedulingDashboard_16372903314670&#47;TCDashboard&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='SchedulingDashboard_16372903314670&#47;TCDashboard' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Sc&#47;SchedulingDashboard_16372903314670&#47;TCDashboard&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en-US' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1637291043343');                    var vizElement = divElement.getElementsByTagName('object')[0];                    if ( divElement.offsetWidth > 800 ) { vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';} else if ( divElement.offsetWidth > 500 ) { vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';} else { vizElement.style.width='100%';vizElement.style.height='1677px';}                     var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>


<h3 align = "left"> Setting Up the Application Worksheet</h3>

<p>I first added a series of four tables with drop down menus to my scheduling application worksheet. The drop down menus allowed Managers to select the information they needed for scheduling from the COUNT table. I updated the COUNT table at 4:30 p.m. each day, when we received the updated count for each active route from our business partner.</p>
<p>&nbsp;</p>

<div align = "center">
<img src="Count_Table1.png" width="673"/>
</div>
<p>&nbsp;</p>

<p>I arranged the COUNT table so that scheduling information could be selected by D0_Cycle, Team, and Day. D0_Cycle referred to the Cycle (collection of routes) that would be activated on the following day. The Cycle number corresponded to the last two digits of the route number (e.g. 1125, 2725 = Cycle 25). When Cycle 25 was on Day 0, for example, Cycle 24 would be on Day 1, Cycle 23 on Day 2, and Cycle 22 on Day 3. Because this relationship between cycle and day was always true, I was able to use D0_Cycle as the criteria for selecting all of the routes that would be active on a given day. Accordingly, the first step in populating the tables in my scheduling app was selecting the D0_Cycle from the drop down menu in the top left corner of the first table.</p>
<p>&nbsp;</p>
    
<div align = "center">
<img src="drop1.png"/>
</div>
<p>&nbsp;</p>

<p>After selecting the D0_Cycle, Managers would select their team from the "Team" drop down menu for each table in the dashboard. Typically, Managers would only need information for their own team for scheduling, but it was good to have the option available to select information for other teams where conflicts for resources arose.</p>
<p>&nbsp;</p>

<div align = "center">
<img src="drop2.png"/>
</div>
<p>&nbsp;</p>
    
<p>Finally, Managers would select the day they wanted to view from the "Day" drop down menu for each table.</p>
<p>&nbsp;</p>
    
<div align = "center">
<img src="drop3.png"/>
</div>
<p>&nbsp;</p>

<p>To populate each table in the app, I needed to write a module that copied data from the COUNT table based on the D0_Cycle, Team, and Day that was selected from the drop down menus for each table. I wrote the "modDropdowns" module (below), which activated a subroutine ("P1") that copied data from the COUNT table and pasted it in the app table.</p> 
<p>&nbsp;</p>

<div align = "center">
<img src="drop_vba1.png" width="800"/>
</div>
<p>&nbsp;</p>
 
<p>I used a for loop and a nested if statement in the subroutine to loop through the COUNT table. The subroutine copied Slot, Route, Est_Time, Meters, and Count data from rows where the D0_Cycle, Team, and Day values were equal to those selected from the drop down menus. The data was then pasted into the relevant table in the app.</p>
<p>&nbsp;</p>
    
<div align = "center">
<img src="drop_vba2.png" width="800"/>
</div>
<p>&nbsp;</p>
    
<p>The next step was adding the scheduling table. I designed the table so that Mangers could assign up to four routes to each of their employees in the order they wanted them completed. When a Manager entered the first route number, the drive time, remaining count, and work minutes remaining on the route automatically populated the subsequent three cells in the row via lookup function. For the remaining routes entered into the schedule, the drive time between routes populated the "Drive" column rather than the drive time from the office to the route.</p>
<p>&nbsp;</p>
    
<div align = "center">
<img src="pop_sch1.gif" width="756"/>
</div>
<p>&nbsp;</p>
  
<h3 align = "left"> Estimating Location & Expected Count </h3>

<p>This portion of the scheduling application was probably the most impactful in terms of changing some perspectives and working towards increasing productivity. Employees were required to provide their Manager with the current count for each of their routes at 11:00 each day, and Managers received the final count for each active route at the end of the day. Without a concrete estimate for how many meters each employee should have read by 11:00 and at the end of the day, Managers struggled to objectively evaluate their employees' performance.</p> 

<p>Poor scheduling was also an issue. The scheduling table provided Managers with drive times and work minutes remaining on routes, which lead to some improvements in scheduling, but under and over scheduling was still a problem. The location and expected count portion of the app helped in this regard by providing Managers with expected count estimates throughout the day that were based on drive times, work rates, and the number of employees scheduled to a route. Based on these estimates, Managers were better able to evaluate and adjust their schedules to maximize efficiency.</p> 
<p>&nbsp;</p>

<div align = "center">
<img src="exp.png" width="760"/>
</div>
<p>&nbsp;</p>
    
<h3 align = "left"> Simple Math </h3> 
    
<p>In the initial version of this app, I focused on providing Managers estimates for the expected count at 11:00 and 4:00. The math for these estimates seemed fairly straightforward based on the employees' daily routine:</p>
<p>&nbsp;</p>

<table>
<tr><th>Task</th><th>Time</th></tr>
<tr><td>Clock in</td><td>7:00 a.m.</td></tr>
<tr><td>Morning meetings</td><td>7:00 a.m. - 7:30 a.m.</td></tr>
<tr><td>Leave for route</td><td>7:30 a.m.</td></tr>
<tr><td>Fuel vehicle</td><td>7:30 a.m. - 7:40 a.m.</td></tr>
<tr><td>Drive to route/Read meters</td><td>7:40 a.m. - 9:00 a.m.</td></tr>
<tr><td>Break</td><td>9:00 a.m. - 9:10 a.m.</td></tr>
<tr><td>Read meters</td><td>9:10 a.m. - 11:00 a.m.</td></tr>
<tr><td>Lunch</td><td>11:00 a.m. - 12:00 p.m.</td></tr>
<tr><td>Read meters</td><td>12:00 p.m. - 2:00 p.m.</td></tr>
<tr><td>Break</td><td>2:00 p.m. - 2:10 p.m.</td></tr>
<tr><td>Read meters/Drive to office</td><td>2:10 p.m. - 4:00 p.m.</td></tr>
<tr><td>Clock out</td><td>4:00 p.m.</td></tr>
</table>
    
<p>&nbsp;</p>
    
<p>The first 30 minutes of the day was spent on morning meetings. Employees typically spent 10 minutes getting gas and took a 10-minute morning break, which left 190 minutes for driving and reading meters. They took an hour lunch, and another 10-minute afternoon break. So, their estimated count at 11:00 would just be 190 - drive time to route(min) * the rate(meters/min) for the route, and 420 - drive time to and from route * the rate for the route. For example, if their drive time was 30 minutes and the rate for their route was 2 meters/min, their estimated count would be 320 at 11:00 and 720 at 4:00. Simple.</p>
<p>&nbsp;</p>

<h3 align = "left"> It Gets Complicated </h3>

<p>If an employee worked alone on a single route, estimating their expected count at 11:00 and 4:00 was simple, but, in reality, employees usually worked on more than one route each day and it was commonplace for more than one employee to work on a route. If an employee finished their first route before 11:00 or 4:00, the time they spent on their first route and the drive time to their second route had to be subtracted from the equation, and their remaining work minutes had to be multiplied by the rate for their second route and added to the count for the first route:</p>
<p>&nbsp;</p>
    
<center><b>Est_Count @11 = R1_Count + ((190 - R1_Min - R1_Drive - R1R2_Drive) * R2_Rate)</b></center>
<p>&nbsp;</p>   
    
<p>The equation expanded in the same manner if they finished their second route and with each additional route:</p>
<p>&nbsp;</p>
    
<center><b>Est_Count @11 = R1_Count + R2_Count + ((190 - R1_Min - R2_Min - R1_Drive - R1R2_Drive - R2R3_Drive) * R3_Rate)</b></center>
<p>&nbsp;</p> 
 
<p>There were also cases where an employee could finish a route, but could not make the drive to their next route before 11:00, or could not make the drive to their next route before needing to head back to the office to clock out at 4:00. That said, I also needed to determine which route would be their last route for the day, because I had to subtract the drive time from that route to the office from the equation for their estimated count at 4:00.</p>
<p>&nbsp;</p>

<div align = "center">
<img src="Cases1.png" width="800"/>
</div>
<p>&nbsp;</p>
    
<p>If more than one employee worked on a route, and they started at the same time, the count and minutes for the route had to be divided by the number of employees working on the route. If they didn't start the route at the same time, I had to determine when each additional employee would arrive, and change the count and minutes each time the number of employees on the route changed. Determining when an employee would arrive at a route was further complicated when that employee had previously worked on a route with one or more other employees.</p>
    
<p>Initially, I used a series of countif and nested if functions to identify cases and calculate estimates. Over time, I modified or added functions as new cases were brought to my attention, but that approach was messy and ultimately not sustainable.</p>
<p>&nbsp;</p>

<div align = "center">
<img src="Mess1.png" width="800"/>
</div>
<p>&nbsp;</p>

<h3 align = "left"> The Solution </h3> 

<p>I used a helper worksheet (SCHEDULE, Sheet4) and wrote a module (modSchedule) to create a solution for the problem. I created a table (i.e. Primary Table) in the worksheet (on the left in the gif below) to store data from the schedule in the App worksheet. I added a column to the table for the start time for each route, which I calculated by adding the minutes spent driving and working prior to the route in question. For example, the start time I calculated for the 2nd route scheduled for Slot 1 (1422) was 139 minutes, which was equal to Slot 1's drive time to route 1 (65 minutes), plus the time they spent working on route 1 (52 minutes), plus their drive time to route 2 (22 minutes). I used the CountIf/Concat Table (on the right below) to identify where more than one reader had been scheduled to work on their 1st; 1st & 2nd; 1st, 2nd, & 3rd; or, 1st, 2nd, 3rd, & 4th routes concurrently. I then used a countif function to divide the minutes for each of these routes by the number of readers working on the route concurrently.</p> 
<p>&nbsp;</p>
    
<div align = "center">
<img src="sch1.gif" width = "800"/>
</div>
<p>&nbsp;</p>
    
<p>The first subroutine (Sub Clear) in the module I created clears the contents of all of the subsequent tables used in the module, and copies the Route, Drive Time, and Minute values from the Primary Table into a second table immediately below (Secondary Table). This second table was necessary because the remaining subroutines in the module recalculate and overwrite the minutes, start, and end times for each Slot-Route combination, and I needed to retain the formulas embedded in the Primary Table that would have been lost when the cells were overwritten.</p>
<p>&nbsp;</p>
    
<p>I wrote the second subroutine (SlotHeading) to create headings for a temporary schedule for each Slot.</p>
<p>&nbsp;</p>
    
<div align = "center">
<img src="sub1.png" width="800"/>
</div>
<p>&nbsp;</p>

<p>I used the data in the Secondary Table (below) to create a temporary schedule for each Slot.</p> 
<p>&nbsp;</p>

<div align = "center">
<img src="2tab.png" width="506"/>
</div>
<p>&nbsp;</p>

<p>I wrote the next sub (TempSch) to create a minute-by-minute (i.e. cell-by-cell) verticle representation of the schedule for each Slot.</p> 
<p>&nbsp;</p>

<p>For example, the temporary schedule for Slot 1 is represented in the first column of the Temporary Schedule (gif below). The second cell in this column represents the first minute (7:30 a.m.) in the schedule for Slot 1. It will be populated with "DTR" for "Drive to Route", along with cells 3 to 66 (i.e. 65-minute drive time), in the final schedule. The 67th cell in the column represents the 66th minute in the schedule for Slot 1 and the start time for their first route. This cell, and cells 68 to 118 (i.e. 52-minutes on route) will be populated with the route number (1441) for the first route assigned to Slot 1. I left the cells that correspond to drive times blank in the Temporary Schedule as I did not need that information in this step. I also plotted each route in its entirety in the Temporary Schedule - the final schedule ends at cell 421, which corresponds to the 420th minute in the schedule (i.e. 4:00 p.m.).</p>
<p>&nbsp;</p>
    
<div align = "center">
<img src="tmpsch.gif" />
</div>
<p>&nbsp;</p>
    
<p>I wrote the UniqueTransP sub to copy and transpose paste a list of the unique routes included in the schedule to use as headings for a Unique Route table.</p>
<p>&nbsp;</p>

<div align = "center">
<img src="sub3.png" width="800"/>
</div>
<p>&nbsp;</p>
 
<p>In the Countdown sub, I loop through the Temporary Schedule and use a countif function to count the number of readers working on the routes in the heading of the Unique Routes table. I multiple the results of the countif function by the rate for the route, and subtract that amount from the count for the route. For each minute in the Temporary Schedule where a route is scheduled and the count is greater than 0, the sub places a "1" in the corresponding cell in the Unique Routes table. By summing the 1's in the Unique Routes table, I was able to recalculate the numbers of minutes needed to complete each route, regardless of the number of readers that worked the route and when they each started.</p>
<p>&nbsp;</p> 

<div align = "center">
<img src="sub4.png" width="800"/>
</div>
<p>&nbsp;</p>
  
<div align = "center">
<img src="u_route_tab.png" width="800"/>
</div>
<p>&nbsp;</p>
    
<p>I wrote the CountMin sub to sum the number of minutes each Reader/Slot should spend on each route. The sub takes into account differences in start times for each reader working on a route, and updates the start times in the Secondary Table for each of their subsequent routes based on the recalculation of the minutes spent on the route in question.</p>
<p>&nbsp;</p> 

<div align = "center">
<img src="sub5.png" width="800"/>
</div>
<p>&nbsp;</p>
    
<p>In the Schedule sub, I created a Schedule Table following the same minute-by-minute process that I used for the Temporary Schedule table. The sub also creates a Count Table, which records the rate for each minute a route is listed in the Schedule Table for each Slot.</p>
<p>&nbsp;</p>    
    
<div align = "center">
<img src="sub6.png" width="800"/>
</div>
<p>&nbsp;</p>
    
<p>The Schedule Table is shown on the left with the corresponding Count Table on the right.</p>
<p>&nbsp;</p> 
    
<div align = "center">
<img src="sch_ct_tab.png" width="800"/>
</div>
<p>&nbsp;</p>
    
<p>I wrote the Trim sub to cut off the Schedule and Count Tables at 420 minutes (e.g. 4:00 p.m.). In the DriveHome sub, I identified the last route each Reader/Slot worked on, looked up the drive time back to the office, and modified the Schedule and Count Tables to account for the time employees would spend driving back to the office.</p>      
<p>&nbsp;</p>
    
<div align = "center">
<img src="sub7.png" width="800"/>
</div>
<p>&nbsp;</p>
    
<p>"DTO" in the cells in the Schedule Table stands for "Drive To Office". The Count Table shows that the rates for the corresponding cells have been deleted.</p>
<p>&nbsp;</p>  
    
<div align = "center">
<img src="dto.png" width="800"/>
</div>
<p>&nbsp;</p>
    
<p>In the Results Table below, I used a lookup function and the Schedule Table to determine the location/task for each Reader/Slot at hourly intervals. I summed the rates from the first minute (cell 2) in the Count Table to the minute corresponding to each hourly interval (e.g. 190 minutes = 11:00 a.m.). </p>   
<p>&nbsp;</p>  
    
<div align = "center">
<img src="results1_tab.png" width="800"/>
</div>
<p>&nbsp;</p>
    
<p>Finally, I wrote the module modClickEvents, which call all of the subs in the Schedule module when a Manager clicks on the Schedule button in the App.</p>
<p>&nbsp;</p>
    
<div align = "center">
<img src="click_mod.png" width="800"/>
</div>
<p>&nbsp;</p>
    
<p>Here's the final product!</p>    
<p>&nbsp;</p>
    
<div align = "center">
<img src="complete3.gif" width="1214"/>
</div>
<p>&nbsp;</p>
    